<a href="https://colab.research.google.com/github/jonathjd/Education-Project/blob/main/5_0_jd_intital_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis Continued

We will continue our EDA of the cost dataset using pandas. We will also be using some Matplotlib as well.

We will be looking at:
1. Descriptive statistics such as mean, trimmed mean, median, mode for institutions which primarily confer:
- Associates degrees
- Bachelors degrees
- Graduate degrees
 both in the US, Northwest region, and Washington (my current state), and for both public, private for-profit, and private non-profit.
 2. We will generate some visualizations such as histograms, boxplots, and bar charts.

## Columns of interest

 1. **Control**:
 - 1: Public
 - 2: Private non-proft
 - 3: Private for-profit
 2. **Region**:
 - 8: Far West (AK, CA, HI, NV, OR, WA)
 3. **HIGHDEG**:
 - 2: Associates degree
 - 3: Bachelors degree
 - 4: Graduate degree 

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/jonathjd/Education-Project/main/data/processed/cleaned_data.csv")
df.head()

In [ ]:
df.describe().transpose()

In [ ]:
# create new feature student spending
df["Student Spending Ratio"] = df["TUITFTE"] / df["INEXPFTE"]

In [ ]:
# Create new df separating all institutions into the degree they primarily grant
us_associates_df = df[df["HIGHDEG"] == 2].copy()
us_associates_df.describe().transpose()

In [ ]:
# trimmed mean
from scipy.stats import trim_mean
trim_mean(us_associates_df["COSTT4_A"], 0.1)

In [ ]:
# Median
us_associates_df["COSTT4_A"].median()

In [ ]:
# mode
us_associates_df["COSTT4_A"].mode()

In [ ]:
us_bachelors_df = df[df["HIGHDEG"] == 3].copy()
us_bachelors_df.describe().transpose()

# Descriptive Statistics

In [ ]:
# trimmed mean
trim_mean(us_bachelors_df["COSTT4_A"], 0.1)

In [ ]:
# median
us_bachelors_df["COSTT4_A"].median()

In [ ]:
# mode
us_bachelors_df["COSTT4_A"].mode()

In [ ]:
us_graduate_df = df[df["HIGHDEG"] == 4].copy()
us_graduate_df.describe().transpose()

In [ ]:
# trimmed mean
trim_mean(us_graduate_df["COSTT4_A"], 0.1)

In [ ]:
# median
us_graduate_df["COSTT4_A"].median()

In [ ]:
# mode
us_graduate_df["COSTT4_A"].mode()

In [ ]:
# private non-profit school that primarily awards graduate degrees
private_graduate_np = us_graduate_df[df["CONTROL"] == 2].copy()

In [ ]:
private_graduate_np.describe().transpose()

In [ ]:
# private for-profit school that primarily awards graduate degrees
private_graduate_fp = us_graduate_df[df["CONTROL"] == 3].copy()

In [ ]:
private_graduate_fp.describe()

In [ ]:
# Public degree that primarily awards bachelors degrees
public_bachelors = us_bachelors_df[df["CONTROL"] == 1].copy()

In [ ]:
public_bachelors.describe()

In [ ]:
Public_Bachelors_Degree = public_bachelors["COSTT4_A"]
Private_Graduate_Degree_np = private_graduate_np["COSTT4_A"]
Private_Graduate_Degree_fp = private_graduate_fp["COSTT4_A"]

# Average Faculty Salary vs Cost of attendance

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.scatterplot(data=df,y=df["AVGFACSAL"], x=df["COSTT4_A"])
ax.set(xlabel="Cost of Attendance (1 year)", ylabel="Average Faculty Salary (Monthly)")

plt.savefig('costta_vs_facsal', dpi=300)

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.scatterplot(data=df,y=df["AVGFACSAL"], x=df["COSTT4_A"], hue=df["CONTROL"])
ax.set(xlabel="Cost of Attendance (1 year)", ylabel="Average Faculty Salary (Monthly)")
plt.legend(labels=["Public", "Private for-profit", "Private non-profit"], loc = 2, bbox_to_anchor = (1,1))

plt.savefig('costta_vs_facsal_hue', dpi=300)

In [ ]:
private_non_profit = df[df["CONTROL"] == 2].copy()

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.scatterplot(data=private_non_profit,y=private_non_profit["AVGFACSAL"], x=private_non_profit["COSTT4_A"])
ax.set(xlabel="Cost of Attendance (1 year)", ylabel="Average Faculty Salary (Monthly)")

# Save image
plt.savefig('costta_vs_facsal_priv', dpi=300)

Is there a correlation/can we model it?

In [ ]:
private_non_profit.corr()

There is a moderate correlation (p=0.61, R2=0.37)

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.regplot(data=private_non_profit,y=private_non_profit["AVGFACSAL"], x=private_non_profit["COSTT4_A"])
ax.set(xlabel="Cost of Attendance (1 year)", ylabel="Average Faculty Salary (Monthly)")

# Linear Regression model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = private_non_profit.COSTT4_A.to_numpy()
Y = private_non_profit.AVGFACSAL.to_numpy()

In [ ]:
X.shape, Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
X_train= X_train.reshape(-1, 1)
Y_train= Y_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

In [ ]:
# import model
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
model = linear_model.LinearRegression()

In [ ]:
model.fit(X_train, Y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
# Calculate MSE & R2
MSE = mean_squared_error(Y_test, Y_pred)
R2 = r2_score(Y_test, Y_pred)
MAE = mean_absolute_error(Y_test, Y_pred)

# model performance
print("Coefficients: ", model.coef_)
print('Intercept: ', model.intercept_)
print(f'Mean square error (MSE): {MSE}')
print(f'Mean Absolute Error (MAE): {MAE}')
print(f"Coefficient of determination (R^2): {R2}")

#Private bachelors vs public bachelors

In [ ]:
# Bar plot contrasting a bachelors degree at a primarily undergraduate program and primarily graduate program that
# is private
sns.set_style('ticks')
sns.set_context('notebook')
fig = plt.figure(figsize=(11.5,5))
ax = sns.barplot(data=[Public_Bachelors_Degree, Private_Graduate_Degree_np, Private_Graduate_Degree_fp], 
                 palette='Blues', 
                 ci="sd", 
                 zorder=5,
                 capsize=.05)
ax.set(xlabel="University Level", ylabel="Cost of Attendance (1 year)")
ax.set_xticklabels(["Public Bachelors Degree", "Private Non-Profit Bachelors Degree", "Private For-Profit Bachelors Degree"])
sns.despine()

plt.savefig('control_barplot')

In [ ]:
# Swarm plot depicting the same data to show distribution
sns.set_style('ticks')
sns.set_context('notebook', font_scale=1.4)
fig = plt.figure(figsize=(9,9))
ax = sns.swarmplot(data=[Public_Bachelors_Degree, Private_Graduate_Degree_np, Private_Graduate_Degree_fp], 
                 palette='Reds')
ax.set(ylabel="Cost of Attendance (1 year)")
ax.set_xticklabels(["Public", "Private non-profit", "Private for-profit"])
sns.despine()

# Save the figure
plt.savefig('swarm_plot', dpi=300)

In [ ]:
sns.set_style('ticks')
sns.set_context('notebook', font_scale=1.5)
fig = plt.figure(figsize=(9,9))
ax = sns.boxplot(data=[Public_Bachelors_Degree, Private_Graduate_Degree_np, Private_Graduate_Degree_fp], 
                 palette='Blues')
ax = sns.swarmplot(data=[Public_Bachelors_Degree, Private_Graduate_Degree_np, Private_Graduate_Degree_fp], 
                 palette='Reds')
ax.set(ylabel="Cost of Attendance (1 year)")
ax.set_xticklabels(["Public", "Private non-profit", "Private for-profit"])
sns.despine()

# save fig
plt.savefig('swarm_plot_boxplot', dpi=300)

# Student spending ratio

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x="Student Spending Ratio")

In [ ]:
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(12,7))
ax = sns.histplot(data=df, x="Student Spending Ratio", hue="HIGHDEG")

In [ ]:
# Compare the student spending between institutions based on the degree they grant
sns.set_style('ticks')
sns.set_context('talk')
fig = plt.figure(figsize=(18,7))
ax = sns.barplot(data=df,
                 x="HIGHDEG",
                 y="Student Spending Ratio",
                 palette='Blues')
sns.despine()

# Miscellaneous Visualizations

In [ ]:

# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.histplot(data=df, x="COSTT4_A", hue="HIGHDEG", bins=40)

# set style
sns.set_style(style="ticks")
sns.despine()


In [ ]:
# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.histplot(data=df, x="COSTT4_A", hue="CONTROL", bins=40)

# set style
sns.set_style(style="ticks")
sns.despine()

In [ ]:
# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.histplot(data=df, x="COSTT4_A", hue="REGION", bins=40)

# set style
sns.set_style(style="ticks")
sns.despine()

- It seems as if there is a higher density of instituions between ~10k and ~35k in region 6 but this isnt clear, let's try and put some numbers to these estimates.

In [ ]:
def proportion(dataframe, col, upper_bound: int, lower_bound: int, hue=None):
  """
  Returns None.

  Keyword Arguments:
  dataframe -- dataframe
  col -- column dataType = str
  upper_bound -- the upper bound
  lower_bound -- the lower bound
  hue --- how to categorize the data dataType = str (default=None)
  """

  # Grab whole subset
  whole_subset = dataframe[(dataframe[col] >= lower_bound) & (dataframe[col] <= upper_bound)]

  if hue == None: # if no hue is defined
    proportion = round(whole_subset[col].count() / df[col].count(), 2) * 100
    print(f"The proportion of the data that falls within the upper and lower bounds is {proportion}%")
  
  else: # if hue is defined
    hue_subset = whole_subset.groupby(by=hue).count()
    for i in range(len(hue_subset)):
      proportion = round(hue_subset[col].iloc[i] / whole_subset[col].count(), 4) * 100
      print(f"The proportion of schools between ${lower_bound} and ${upper_bound} in {hue} {i} is:")
      print(f"{proportion}%")
      print("\n")
    return hue_subset[col]

In [ ]:
proportion(df, col="COSTT4_A", lower_bound=10000, upper_bound=25000, hue="REGION")

Region 5 (**Southeast (AL, AR, FL, GA, KY, LA, MS, NC, SC, TN, VA, WV**) has the highest proportion of schools with a cost of tuition between 10k and 35k

- 25% of the schools in region 5 are between 10k and 35k per year.

In [ ]:
# using control as the hue
proportion(df, col="COSTT4_A", lower_bound=20000, upper_bound=25000, hue="CONTROL")

- More than half of the public universities in the US cost between 10k and 35k for attendance for 1 year

In [ ]:
# using high degree as the hue
proportion(df, col="COSTT4_A", lower_bound=20000, upper_bound=25000, hue="HIGHDEG")

In [ ]:
# using stabbr as hue
proportion(df, col="COSTT4_A", lower_bound=10000, upper_bound=20000, hue="STABBR")

### Descriptives:
The mean/median/mode cost of attendance for one year at an instituiton which primarily confers:
1. Associates degrees: 17,667 / 14,379 / 11,349
2. Bachelors degrees: 29,529 / $25,406 / 23,776
3. Graduate degrees: 34,549 / 29,990 / 28,149

In [ ]:
us_associates_df["COSTT4_A"].hist(bins=50)

In [ ]:
us_bachelors_df["COSTT4_A"].hist(bins=40)

In [ ]:
us_graduate_df["COSTT4_A"].hist(bins=40)

In [ ]:
#What is the variance in the cost of attendance?
from statistics import variance

variance(us_associates_df["COSTT4_A"])

In [ ]:
variance(us_bachelors_df["COSTT4_A"])

In [ ]:
variance(us_graduate_df["COSTT4_A"])

In [ ]:
#Standard deviation
from statistics import stdev

stdev(us_associates_df["COSTT4_A"])

In [ ]:
stdev(us_bachelors_df["COSTT4_A"])

In [ ]:
stdev(us_graduate_df["COSTT4_A"])

In [ ]:
# Mean absolute deviation
import numpy as np

np.mean(np.absolute(us_associates_df["COSTT4_A"] - np.mean(us_associates_df["COSTT4_A"])))

In [ ]:
# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.histplot(data=df, x="COSTT4_A", hue="HIGHDEG", kde=True)

# set style
sns.set_style(style="ticks")
sns.despine()

In [ ]:
# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.boxplot(data=df, x="REGION", y="COSTT4_A")

# set style
sns.set_style(style="ticks")
sns.despine()

In [ ]:
# make figure
fig = plt.figure(figsize=(12,7))
fig = sns.boxplot(data=df, x="HIGHDEG", y="COSTT4_A")

# set style
sns.set_style(style="ticks")
sns.despine()


- it looks like instituitions which primarily award associates degrees are heavily positively skewed